# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [28]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

# %matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [29]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.1
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Create Model and Load Trained Weights

In [30]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
# print(COCO_MODEL_PATH)
# print(type(COCO_MODEL_PATH))
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [31]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [35]:
# # Load a random image from the images folder
# import cv2

# #####edit#####

# def save_images_from_folder(folder):
#     images = []
#     for filename in os.listdir(folder):
#         img = cv2.imread(os.path.join(folder,filename))
#         if img is not None:
#             images = np.append(img)
#     x = '/home/natanan/catkin_ws/src/Mask_RCNN/img_saved/{}'.format(filename)
#     cv2.imwrite(x,images)
#     return images 

# ##############################################################################################

# file_names = next(os.walk(IMAGE_DIR))[2]
# image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# # Run detection
# results = model.detect([image], verbose=1)

# # Visualize results
# # r = results[0]

# # #rois = bounding box
# # #masks = mask area
# # #class_ids = class number of type ex sheep's class is 19

# # visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
# #                             class_names, r['scores'])

# ##########################################################################################################
# # sua
# # # for first image (as we're providing single image only)
# r = results[0]

# # get indices corresponding to unwanted classes
# indices_to_remove  = np.where((r['class_ids'] != 1) & (r['class_ids'] != 3)) #output index


# # remove corresponding entries from `r`
# # axis 0 -> delete row
# # axis 1 -> delete column
# # axis 2 -> delete 3D
# new_class_ids = np.delete(r['class_ids'], indices_to_remove, axis=0)
# new_rois = np.delete(r['rois'], indices_to_remove, axis=0)
# new_scores = np.delete(r['scores'], indices_to_remove, axis=0)
# new_masks = np.delete(r['masks'], indices_to_remove, axis=2)

# # Get Mask pixels
# x=[]
# y=[]
# y_image_axis = new_masks.shape[0]
# x_image_axis = new_masks.shape[1]
# All_instance = new_masks.shape[2]

# image_rgba = cv2.cvtColor(image,cv2.COLOR_RGB2RGBA)

# if All_instance > 0:
#     for i in range(y_image_axis): #y-axis
#         for j in range(x_image_axis): #x-axis
#             for k in range(All_instance):
#                 if new_masks[i,j,k] == True: #i - vertical , j - horizontal
#                     image_rgba[i,j,3] = 0
#                     x.append(j)
#                     y.append(i)
#     check = 1
# else : 
#     print('No require instance')
#     check = 0

# # cv2.imshow('rgba', image_rgba)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()

# # if check != 0:
# #     x1 = x[::-1]
# #     y1 = y #[::-1]
# #     plt.scatter(x1, y1)
# #     plt.show()

# # img[y[:],x[:]] = [255,255,255]

# # print(x1)
# # print(y1)


# visualize.display_instances(image_rgba, new_rois, new_masks, new_class_ids, class_names,show_bbox=False)


############################### NEW EDIT #####################################################
# Load a random image from the images folder
import cv2

#####edit#####
s = 0
Path = '/home/natanan/catkin_ws/src/Mask_RCNN'
Input_img_dir = '/images'
Output_img_dir = '/img_saved'

RGB_input_DIR = '/media/natanan/CITYSCAPE/testrtab/RGB'
Depth_input_DIR = '/media/natanan/CITYSCAPE/testrtab/depth'
def save_images_from_path_folder(RGB_input_DIR, Depth_input_DIR,s):
    for filename in os.listdir(RGB_input_DIR):
        image = cv2.imread(os.path.join(RGB_input_DIR+'/'+filename))
        depth = cv2.imread(os.path.join('/media/natanan/CITYSCAPE/testrtab/depth/depth_{}.png'.format(s)))
        results = model.detect([image], verbose=1)
        r = results[0]
        indices_to_remove  = np.where((r['class_ids'] != 1) & (r['class_ids'] != 3)) #output index
        new_class_ids = np.delete(r['class_ids'], indices_to_remove, axis=0)
        new_rois = np.delete(r['rois'], indices_to_remove, axis=0)
        new_scores = np.delete(r['scores'], indices_to_remove, axis=0)
        new_masks = np.delete(r['masks'], indices_to_remove, axis=2)
        # Get Mask pixels
        x=[]
        y=[]
        y_image_axis = new_masks.shape[0]
        x_image_axis = new_masks.shape[1]
        All_instance = new_masks.shape[2]
        image_rgba = cv2.cvtColor(image,cv2.COLOR_RGB2RGBA)
        depth_rgba = cv2.cvtColor(depth, cv2.COLOR_RGB2RGBA)

        if All_instance > 0:
            for i in range(y_image_axis): #y-axis
                for j in range(x_image_axis): #x-axis
                    for k in range(All_instance):
                        if new_masks[i,j,k] == True: #i - vertical , j - horizontal
                            image_rgba[i,j,3] = 0
                            depth_rgba[i,j,3] = 0
                            x.append(j)
                            y.append(i)
            check = 1
        else : 
            print('No require instance')
            check = 0

        out = '/home/natanan/catkin_ws/src/Mask_RCNN/img_saved/for_RTAB/RGB/RGB_{}.png'.format(s)  #filename[0:len(filename)-4] to remove .jpg
        depth_out = '/home/natanan/catkin_ws/src/Mask_RCNN/img_saved/for_RTAB/Depth/Depth_{}.png'.format(s)
        cv2.imwrite(out,image_rgba)
        cv2.imwrite(depth_out,depth)

        s  = s + 1

    return  image_rgba, new_rois, new_masks, new_class_ids

image_rgba, new_rois, new_masks, new_class_ids = save_images_from_path_folder(RGB_input_DIR, Depth_input_DIR,s)

# visualize.display_instances(image_rgba, new_rois, new_masks, new_class_ids, class_names,show_bbox=False)

Processing 1 images
image                    shape: (480, 640, 3)         min:    0.00000  max:  131.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   13.20000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
No require instance
Processing 1 images
image                    shape: (480, 640, 3)         min:    0.00000  max:  226.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  105.20000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
No require instance
Processing 1 images
image                    shape: (480, 640, 3)         min:    0.00000  max:  240.00000  uint8
molded_images            shape: (1, 1024, 

IndexError: index 3 is out of bounds for axis 2 with size 3

25
g
12283150_12d37e6389_z
